# Case Study 1: The Ancient Greek Pottery - Classification

Given the fabric, shape name, provenance, inscription, there or not and  the technique predict date <br>
Will use some sort of random forest here

In [1]:
#reading the data
import pandas as pd
df_orig = pd.read_csv('Pottery_data.csv')

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (14,20,24,25,26,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
df_orig.head(None)

,URI,Vase Number,Fabric,Technique,Sub Technique,Shape Name,Provenance,Date,Inscriptions,Attributed To,...,Technical Observations,Image Credits,Pleiades URI,Latitude,Longitude,LIMC ID,LIMC Web,British Museum Link,Ure Museum Link,Bryn Mawr College Link
0,http://www.beazley.ox.ac.uk/record/AA1CD952-92...,1.0,ATHENIAN,BLACK-FIGURE,NaN,"AMPHORA, NECK","ITALY, ETRURIA, VULCI",-525 to -475,NaN,CHIUSI P by KUNZE-GOTTE | LEAGROS GROUP by KUN...,...,NaN,NaN,https://pleiades.stoa.org/places/413393,42.421429,11.702499,6368,http://ark.dasch.swiss/ark:/72163/080e-73fdcff...,NaN,NaN,NaN
1,http://www.beazley.ox.ac.uk/record/85D73CAF-DC...,2.0,ATHENIAN,BLACK-FIGURE,NaN,"KRATER, COLUMN FRAGMENT","GREECE, ATHENS, AGORA",-550 to -500,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,http://www.beazley.ox.ac.uk/record/B7867649-B6...,3.0,ATHENIAN,BLACK-FIGURE,NaN,"AMPHORA, NECK","ITALY, ETRURIA, VULCI",-550 to -500,NaN,NaN,...,NaN,NaN,https://pleiades.stoa.org/places/413393,42.421429,11.702499,1023,http://ark.dasch.swiss/ark:/72163/080e-73b1f46...,NaN,NaN,NaN
3,http://www.beazley.ox.ac.uk/record/64B75012-4B...,4.0,ATHENIAN,BLACK-FIGURE,NaN,"AMPHORA, NECK",NaN,-550 to -500,NaN,<em>Near</em> MEDEA GROUP by UNKNOWN,...,NaN,NaN,NaN,NaN,NaN,1009,http://ark.dasch.swiss/ark:/72163/080e-73b1c12...,NaN,NaN,NaN
4,http://www.beazley.ox.ac.uk/record/ED45A6A0-5C...,5.0,ATHENIAN,BLACK-FIGURE,NaN,"AMPHORA, NECK",NaN,-525 to -475,NaN,ANTIMENEAN by KUNZE-GOTTE | DAYBREAK P by POUI...,...,NaN,NaN,NaN,NaN,NaN,10571,http://ark.dasch.swiss/ark:/72163/080e-74394cd...,https://www.britishmuseum.org/collection/objec...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131192,http://www.beazley.ox.ac.uk/record/C7AFB93A-48...,9053423.0,ATHENIAN,RED-FIGURE,NaN,CUP FRAGMENT,NaN,-500 to -450,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131193,http://www.beazley.ox.ac.uk/record/49E67A19-A2...,9053424.0,ATHENIAN,RED-FIGURE,NaN,CUP FRAGMENT,NaN,-500 to -450,NaN,CLINIC P by BOTHMER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131194,http://www.beazley.ox.ac.uk/record/8D202FC3-F4...,9053425.0,ATHENIAN,RED-FIGURE,NaN,CUP FRAGMENT,NaN,-500 to -450,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131195,http://www.beazley.ox.ac.uk/record/E0421614-B6...,9053426.0,ATHENIAN,RED-FIGURE,NaN,CUP FRAGMENT,NaN,-525 to -475,NaN,<em>School of</em> MAKRON by BOTHMER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
#dropping columns that do not contribute directly
df_orig = df_orig[['Vase Number','Fabric','Technique','Shape Name','Provenance','Date','Inscriptions',
       ' Decoration']]

In [3]:
df_orig.columns #updated column list

Index(['URI', 'Vase Number', 'Fabric', 'Technique', 'Sub Technique',
       'Shape Name', 'Provenance', 'Date', 'Inscriptions', ' Attributed To',
       ' Decoration', ' Collection Record', ' Publication Record',
       ' Inscriptions II', 'Measurements', 'Volume', 'Weight', 'Restorations',
       'Ornament', ' Technical Observations', 'Image Credits', 'Pleiades URI',
       'Latitude', 'Longitude', 'LIMC ID', 'LIMC Web', 'British Museum Link',
       'Ure Museum Link', 'Bryn Mawr College Link'],
      dtype='object')

In [58]:
df_orig.shape

(131197, 8)

This is the original raw relevant dataset. The problem statemnet here is do location based clustering of the artefacts. classify to find the time period. association rule mining between techniques and shapes.

In [59]:
df_orig.isna().sum()

Vase Number          1
Fabric             446
Technique        13640
Shape Name        2013
Provenance       76427
Date             33809
Inscriptions    126850
 Decoration      37397
dtype: int64

In [61]:
#for inscription, if it exists, we write 1 and if it doesnt we write 0
df_orig['Inscriptions'] = df_orig['Inscriptions'].apply(lambda x: 1 if pd.notna(x) else 0)

<ipython-input-61-0693bd376e06>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orig['Inscriptions'] = df_orig['Inscriptions'].apply(lambda x: 1 if pd.notna(x) else 0)


In [62]:
df_orig.isna().sum()

Vase Number         1
Fabric            446
Technique       13640
Shape Name       2013
Provenance      76427
Date            33809
Inscriptions        0
 Decoration     37397
dtype: int64

In [64]:
df_orig['Provenance'].nunique()

1694

In [66]:
df_orig.dropna(inplace = True)

<ipython-input-66-a4506647405c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orig.dropna(inplace = True)


In [67]:
df_orig.isna().sum()

Vase Number     0
Fabric          0
Technique       0
Shape Name      0
Provenance      0
Date            0
Inscriptions    0
 Decoration     0
dtype: int64

In [68]:
df_orig['Provenance'].nunique()

1607

In [69]:
df_orig.shape

(46045, 8)

In [70]:
df_orig.to_csv('Cleaned_Pottery_data.csv') #start from here next time

# Case Study 2: The SARD Database - Clustering

Given the columns ' Site', 'Site code', 'DecdegS', 'DecdegE', 'Country','Province or district', 'Biome', 'Dating technique', 'Material dated','Uncertainty', 'Archaeological Period','Site Type' cluster the data and then plot the clusters, find how and why clustering happened and create a geographic plot and talk about stuff <br>
Will use DB Scan here

In [19]:
df_SARD = pd.read_csv('SARD_Mar2021_14C.csv')

In [20]:
df_SARD.head()

,Site,Site code,DecdegS,DecdegE,Country,Province or district,RFZ,Biome,Stratigraphic context,Dating technique,...,Lab ID,Date,Uncertainty,Archaeological Period,Archaeological Sub-chronology,Sample context,Site Type,delta 13,refcode,Notes
0,/hei-/khomas,HKO,-28.583,17.083,South Africa,Northern Cape,W,Succulent Karoo Biome,"Area 3, Y21, Layer Unit 2, 7 cm depth",conventional 14C,...,Pta-5444,106,60,LSA,Ceramic LSA,NaN,open site,-23.2,webley1992hap,Also called /Ai tomas
1,/hei-/khomas,HKO,-28.583,17.083,South Africa,Northern Cape,W,Succulent Karoo Biome,"Area 2, P32, LayerSLS2, 16 cm depth",conventional 14C,...,Pta-5452,330,45,LSA,Ceramic LSA,NaN,open site,-19.7,webley1992hap,Also called /Ai tomas
2,/hei-/khomas,HKO,-28.583,17.083,South Africa,Northern Cape,W,Succulent Karoo Biome,"Area 1, N33, Layer BL, 5 cm depth",conventional 14C,...,Pta-5458,420,50,LSA,Ceramic LSA,NaN,open site,-22.3,webley1992hap,Also called /Ai tomas
3,/hei-/khomas,HKO,-28.583,17.083,South Africa,Northern Cape,W,Succulent Karoo Biome,"Area 3, Y21, Layer LBS3, 18 cm depth",conventional 14C,...,Pta-5530,1980,80,LSA,Ceramic LSA,NaN,open site,-19.7,webley1992hap,Also called /Ai tomas
4,Abrahamsdam,ABH,-28.508,20.610,South Africa,Northern Cape,S,Savanna Biome,Human burial NMB 1405,conventional 14C,...,Pta-2905,360,50,NaN,NaN,NaN,Human burial,NaN,morris1992scs,NaN


In [21]:
df_SARD.columns

Index([' Site', 'Site code', 'DecdegS', 'DecdegE', 'Country',
       'Province or district', 'RFZ', 'Biome', 'Stratigraphic context',
       'Dating technique', 'Material dated', ' Species', 'Lab ID', 'Date',
       'Uncertainty', 'Archaeological Period', 'Archaeological Sub-chronology',
       'Sample context', 'Site Type', 'delta 13', 'refcode', 'Notes'],
      dtype='object')

In [30]:
df_SARD['delta 13'].nunique()

191

In [23]:
df_SARD.shape

(2907, 22)

In [24]:
df_SARD.isna().sum()

 Site                             179
Site code                         179
DecdegS                           179
DecdegE                           179
Country                           179
Province or district              179
RFZ                               186
Biome                             305
Stratigraphic context            1018
Dating technique                  179
Material dated                    712
 Species                         2503
Lab ID                            179
Date                              179
Uncertainty                       206
Archaeological Period             360
Archaeological Sub-chronology    1499
Sample context                   2744
Site Type                         359
delta 13                         2434
refcode                           179
Notes                            2017
dtype: int64

In [ ]:
#taking only [' Site', 'Site code', 'DecdegS', 'DecdegE', 'Country','Province or district', 'Biome', 'Dating technique', 'Material dated','Uncertainty', 'Archaeological Period','Site Type']

In [34]:
df_SARD = df_SARD[[' Site', 'Site code', 'DecdegS', 'DecdegE', 'Country','Province or district', 'Biome', 'Dating technique', 'Material dated','Uncertainty', 'Archaeological Period','Site Type']]

In [35]:
df_SARD.head()

,Site,Site code,DecdegS,DecdegE,Country,Province or district,Biome,Dating technique,Material dated,Uncertainty,Archaeological Period,Site Type
0,/hei-/khomas,HKO,-28.583,17.083,South Africa,Northern Cape,Succulent Karoo Biome,conventional 14C,charcoal,60,LSA,open site
1,/hei-/khomas,HKO,-28.583,17.083,South Africa,Northern Cape,Succulent Karoo Biome,conventional 14C,charcoal,45,LSA,open site
2,/hei-/khomas,HKO,-28.583,17.083,South Africa,Northern Cape,Succulent Karoo Biome,conventional 14C,charcoal,50,LSA,open site
3,/hei-/khomas,HKO,-28.583,17.083,South Africa,Northern Cape,Succulent Karoo Biome,conventional 14C,charred bone,80,LSA,open site
4,Abrahamsdam,ABH,-28.508,20.610,South Africa,Northern Cape,Savanna Biome,conventional 14C,bone,50,NaN,Human burial


In [41]:
df_SARD.dropna(inplace=True)

In [42]:
df_SARD.isna().sum()

 Site                    0
Site code                0
DecdegS                  0
DecdegE                  0
Country                  0
Province or district     0
Biome                    0
Dating technique         0
Material dated           0
Uncertainty              0
Archaeological Period    0
Site Type                0
dtype: int64

In [43]:
df_SARD.shape

(1834, 12)

In [71]:
df_SARD.to_csv('Cleaned_SARD_data.csv') #start from here next time

# Case Study 3: The Grave Data - Association Rule Mining

Time buckets - > Location ->  and types of burial goods found
Think 2016-18 -> target -> items purchasef <br>
Will use Apriori algorithm 